In [206]:
import generator
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Activation,LSTM,Bidirectional,Embedding,Conv2D,BatchNormalization,Activation,Masking,LeakyReLU
from tensorflow.keras.activations import tanh,relu
from tensorflow.keras.models import Model
from tensorflow.keras import Model as models
import numpy as np
import librosa
import matplotlib.pyplot as plt
import logging, os
from sklearn.metrics.pairwise import cosine_similarity

logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from IPython.display import display
from time import sleep

print('Test 1')
dh = display('Test2',display_id=True)
sleep(1)
dh.update('Test3')

files , json,json_word = generator.gen()
maxi = 1021

def expdim(x,axis):
    return tf.expand_dims(x,axis=axis)

Test 1


'Test3'

In [128]:
#공식 구해야함
'''
maxi = 0
for k,i in enumerate(files):
    wav,_ = librosa.load(files[i]['file_path'],sr=16000)
    mfcc = librosa.feature.mfcc(wav,n_mfcc=40)
    if mfcc.shape[1] > maxi:
        maxi = mfcc.shape[1]
    if k%200==0:
        print(k)
'''


emb_layer = Embedding(len(json),512)
opt = tf.keras.optimizers.Adam(10e-3)
maxi = 1021
sos_emb = emb_layer(1)

emb_results = [None]*len(files)
mfccs_temp = []
#전체 파일 생성
files_word_num = []

for k,i in enumerate(files):
    if k % 20 == 0:
        print(tf.cast([ json[j][0] for j in files[i]['file_txt']],tf.float32))
    file_word_num = tf.cast([ json[j][0] for j in files[i]['file_txt']],tf.float32)
    files_word_num.append(file_word_num)
    wav,_ = librosa.load(files[i]['file_path'],sr=16000)
    emb_result = emb_layer(file_word_num)
    '''
    emb_result = []
    for word in file_word_num:
        print(word)
        if word == 0:
            emb_result.append(tf.zeros([1,512]))
        else:
            emb_result.append(tf.expand_dims(emb_layer(word),axis=0))
    emb_result = tf.concat(emb_result,axis=0)'''
    
    
    emb_results[k] = emb_result
    mfcc = np.expand_dims(
                np.expand_dims(
                    librosa.feature.mfcc(wav,n_mfcc=40),axis=-1),axis=0)

    tak = np.zeros((1,40,maxi-mfcc.shape[-2],1),dtype=np.float32)
    mfcc = np.concatenate([mfcc,tak],axis=2)
    mfccs_temp.append(mfcc)
mfccs = np.concatenate(mfccs_temp,axis=0)
    
max_len_txt = 0
for i in emb_results:
    if max_len_txt < i.shape[0]:
        max_len_txt = i.shape[0]

embs = []
for k,i in enumerate(emb_results):
    if k % 20 == 0:
        print(k)
    emb = np.zeros((max_len_txt-(i.shape[0]),512),dtype=np.float32)
    padded_emb = np.expand_dims(np.concatenate([i,emb],axis=0),axis=0)
    embs.append(padded_emb)
embs = np.concatenate(embs,axis=0)

train_mfccs = mfccs[:-270]    
test_mfccs = mfccs[-270:]

train_embs = embs[:-270]    
test_embs = embs[-270:]

train_len = files_word_num[:-270]
test_len = files_word_num[-270:]

json_emb = tf.cast([ json[i][0] for i in json],tf.float32)

tf.Tensor(
[ 1.  3.  4.  5.  6.  7.  8.  9.  7. 10. 11. 12. 13.  7. 14.  7. 15.  7.
 16. 17.  7. 18. 19. 20. 21.  7. 22. 23. 24. 25. 26. 27. 28. 29. 30.  7.
 31.  9.  7. 32. 33.  2.], shape=(42,), dtype=float32)
tf.Tensor(
[  1. 378.   7. 379.   9.   7. 380. 381.  91. 382. 383.  77. 384. 385.
 156. 292. 386. 387.  17.  30.   7. 388.   9. 389. 390. 391.  91.  12.
 392.   7. 393. 394.   7. 395. 396.  46.   7. 397.  77.   7. 398.   2.], shape=(42,), dtype=float32)
tf.Tensor(
[  1.   7. 617.  91. 618.  91.  80. 619. 156.   7. 620.  46.   7. 621.
  91. 103. 622.  77. 623.   7. 624. 540. 625. 626.  27. 107.  75.  73.
 627. 628.  46. 629. 415. 630.   7. 631.   9.   7. 632. 227. 633. 519.
 262.   7. 634.   9.   7.  45.   2.], shape=(49,), dtype=float32)
tf.Tensor(
[  1. 855. 736. 586.  46. 856. 857. 318. 858.  30.   7. 859.   9.   7.
 603. 860. 861. 862. 153.  42. 733. 863.  77. 864. 865.   2.], shape=(26,), dtype=float32)
tf.Tensor(
[1.000e+00 7.000e+00 5.880e+02 1.031e+03 2.700e+01 7.790e+02

tf.Tensor(
[1.000e+00 2.000e+01 6.800e+02 9.100e+01 1.863e+03 1.520e+02 7.700e+01
 9.700e+02 1.700e+01 9.100e+01 3.830e+02 5.000e+01 5.100e+01 3.000e+01
 2.824e+03 7.520e+02 7.000e+00 1.333e+03 9.000e+00 2.152e+03 4.600e+01
 5.000e+01 7.330e+02 1.700e+01 7.000e+00 3.253e+03 9.000e+00 7.000e+00
 4.390e+02 1.580e+02 5.100e+01 3.254e+03 3.000e+01 7.300e+01 3.255e+03
 2.000e+00], shape=(36,), dtype=float32)
tf.Tensor(
[1.000e+00 8.970e+02 3.251e+03 2.270e+02 7.920e+02 1.308e+03 1.700e+01
 2.950e+03 8.000e+01 2.950e+03 4.600e+01 1.700e+01 3.303e+03 2.300e+01
 2.050e+02 2.950e+03 6.760e+02 6.450e+02 3.304e+03 2.000e+01 4.250e+02
 4.600e+01 1.926e+03 6.560e+02 1.230e+03 7.300e+01 2.590e+02 3.298e+03
 2.000e+00], shape=(29,), dtype=float32)
tf.Tensor(
[1.000e+00 1.191e+03 3.360e+03 4.890e+02 7.740e+02 2.050e+02 1.929e+03
 7.000e+00 8.690e+02 4.250e+02 8.960e+02 2.947e+03 2.000e+00], shape=(13,), dtype=float32)
tf.Tensor(
[1.000e+00 9.300e+01 4.250e+02 9.100e+01 9.720e+02 7.500e+01 7.300e+01
 1

tf.Tensor(
[1.000e+00 4.900e+02 1.165e+03 5.062e+03 4.471e+03 7.700e+01 2.080e+03
 4.370e+02 1.700e+01 2.388e+03 6.700e+02 4.600e+01 4.900e+02 1.030e+02
 1.050e+02 2.620e+02 1.970e+02 9.700e+02 2.970e+02 1.618e+03 8.510e+02
 6.800e+02 4.150e+02 1.063e+03 2.000e+00], shape=(25,), dtype=float32)
tf.Tensor(
[1.000e+00 7.000e+00 3.258e+03 3.000e+01 2.706e+03 5.112e+03 1.219e+03
 1.103e+03 3.000e+01 3.000e+00 8.290e+02 8.060e+02 7.000e+00 5.098e+03
 4.680e+02 4.600e+01 3.800e+01 6.000e+00 7.000e+00 6.800e+02 6.120e+02
 5.113e+03 2.600e+01 7.000e+00 5.095e+03 1.560e+02 7.300e+01 5.097e+03
 2.000e+00], shape=(29,), dtype=float32)
tf.Tensor(
[1.000e+00 5.138e+03 5.131e+03 5.132e+03 1.030e+02 4.280e+03 7.300e+01
 5.139e+03 7.920e+02 2.000e+00], shape=(10,), dtype=float32)
tf.Tensor(
[1.000e+00 4.423e+03 1.700e+01 5.201e+03 2.057e+03 5.206e+03 7.000e+00
 5.207e+03 5.150e+02 5.208e+03 3.000e+00 5.209e+03 7.700e+01 5.210e+03
 7.000e+00 5.211e+03 9.000e+00 7.000e+00 2.017e+03 8.650e+02 4.600e+01
 4

tf.Tensor(
[1.000e+00 8.600e+02 4.160e+02 4.870e+02 2.730e+02 1.460e+02 8.000e+01
 3.000e+00 1.997e+03 6.557e+03 6.000e+00 5.421e+03 1.190e+02 7.720e+02
 5.560e+02 7.480e+02 7.700e+01 2.378e+03 7.000e+00 6.558e+03 8.310e+02
 2.000e+00], shape=(22,), dtype=float32)
tf.Tensor(
[1.000e+00 7.000e+00 1.438e+03 9.000e+00 7.000e+00 6.608e+03 4.849e+03
 2.250e+02 1.700e+01 2.300e+01 9.300e+01 3.910e+02 6.110e+02 4.600e+01
 5.420e+02 1.900e+01 5.600e+01 3.313e+03 1.700e+01 1.030e+02 4.914e+03
 3.000e+00 3.033e+03 9.000e+00 6.609e+03 1.030e+02 1.050e+02 6.850e+02
 6.610e+03 1.542e+03 3.000e+01 7.000e+00 8.960e+02 6.000e+00 6.611e+03
 9.300e+01 7.000e+00 1.233e+03 3.800e+01 3.263e+03 6.612e+03 9.110e+02
 2.000e+00], shape=(43,), dtype=float32)
tf.Tensor(
[1.000e+00 1.560e+02 3.000e+00 6.653e+03 7.000e+00 8.370e+02 4.249e+03
 6.654e+03 6.655e+03 7.700e+01 1.428e+03 7.000e+00 3.925e+03 1.971e+03
 4.600e+01 2.301e+03 3.000e+01 1.200e+03 7.000e+00 1.760e+03 5.659e+03
 6.656e+03 6.657e+03 1.680e+02 9.

tf.Tensor(
[1.000e+00 7.793e+03 7.862e+03 7.700e+01 2.700e+01 7.364e+03 3.287e+03
 7.000e+00 6.000e+01 1.906e+03 9.000e+00 7.863e+03 2.000e+00], shape=(13,), dtype=float32)
tf.Tensor(
[1.000e+00 1.460e+02 6.980e+02 4.686e+03 2.500e+01 7.967e+03 7.300e+01
 5.420e+02 3.303e+03 4.600e+01 3.000e+00 2.870e+02 6.842e+03 2.000e+00], shape=(14,), dtype=float32)
tf.Tensor(
[1.000e+00 9.100e+01 8.041e+03 3.000e+00 8.026e+03 8.042e+03 3.000e+01
 3.000e+00 8.043e+03 7.550e+02 2.000e+00], shape=(11,), dtype=float32)
tf.Tensor(
[1.000e+00 4.900e+02 4.910e+02 1.561e+03 3.000e+01 7.000e+00 8.066e+03
 2.163e+03 4.600e+01 6.081e+03 7.030e+02 3.386e+03 2.000e+00], shape=(13,), dtype=float32)
tf.Tensor(
[1.000e+00 7.030e+02 1.570e+02 1.200e+01 3.280e+02 7.000e+00 8.099e+03
 6.280e+02 7.030e+02 1.570e+02 8.100e+03 6.000e+00 8.101e+03 2.620e+02
 9.190e+02 2.620e+02 4.890e+02 8.540e+02 5.900e+01 4.460e+02 9.100e+01
 8.102e+03 4.890e+02 2.620e+02 4.900e+02 8.540e+02 7.700e+01 7.900e+02
 4.250e+02 1.029e+03 8.

In [6]:
comp = tf.cast([ json[j][0] for j in files['1673-143396-0002']['file_txt']],tf.float32)
compre = emb_layer(comp)
comp = emb_layer(file_word_num)

In [7]:
cosine_similarity(expdim(comp[0],0),expdim(compre[0],0))

array([[1.]], dtype=float32)

In [177]:
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

def emb_to_word(target):
    cosine_result = cosine_similarity(emb_layer(json_emb),target)
    result = []
    for i in range(cosine_result.shape[1]):
        table = cosine_result[:,i].tolist()
        result.append(json_word[table.index(max(table))])
    return result

def hard_tanh(x):
    return tf.minimum(20.0, tf.maximum(0.0, x)) 
    
get_custom_objects().update({'hard_tanh':Activation(hard_tanh)})
    
class encoder(Model):
    def conv_init(self):
        conv = Sequential([
            Conv2D(32,(21,81),strides=(2,2),padding='valid',input_shape=(40,1021,1)),
            BatchNormalization(-1),
            Activation(hard_tanh),
            Conv2D(32,(10,41),strides=(1,2),padding='valid'),
            BatchNormalization(-1),
            Activation(hard_tanh),
        ])
        
        return conv
    
    def __init__(self):
        super(encoder,self).__init__()
        #self.conv = self.conv_init()
        self.rnn = Sequential([
            Masking(mask_value=0.0),
            LSTM(256,input_shape=(None,40)),
        ])
        
    def call(self,audio):
        audio = tf.transpose(tf.squeeze(audio,3),[0,2,1])
        #x = self.conv(audio)
        #x = self.rnn(tf.squeeze(x,1))
        x = self.rnn(audio)
        return x
        
class seq2(Model):
    def __init__(self, enc, dec):
        super(seq2,self).__init__()
        self.enc = enc
        self.dec = dec
        
    def call(self,audio, txt_emb, txt_len):
        enc_out = self.enc(audio)
        dec_out = self.dec(enc_out,txt_emb, txt_len)
        
        return dec_out
    
    def predict(self, audio, sos_emb):
        
        enc_out = self.enc(audio)
        enc_out = tf.expand_dims(enc_out,axis=0)
        
        sos_emb = tf.expand_dims(tf.expand_dims(sos_emb,axis=0),axis=0)
        
        predict_emb = tf.concat([sos_emb,enc_out],axis=2)
        
        #init
        #predict_emb = self.dec.mask(predict_emb)
        output, h , c = self.dec.rnn(predict_emb)
        cur_word = tf.expand_dims(tf.expand_dims(output[0,-1,:],axis=0),axis=0)
        output_dim = output
        
        hidden = [h,c]
        stop_sign = False
        result = [sos_emb]
        
        k=0
        while not stop_sign:
            
            if k == 3:
                break
            enc_outs = tf.tile(enc_out,[1,output_dim.shape[1],1])
            predict_emb = tf.concat([output_dim,enc_outs],axis=2)
            
            #predict_emb = tf.concat([cur_word,enc_out],axis=2)
            
            #predict_emb = self.dec.mask(tf.concat([cur_word,enc_out],axis=2))
            
            print(predict_emb.shape)
            output , h ,c = self.dec.rnn(
                predict_emb,
                initial_state = hidden
            )
            hidden = [h,c]
            
            pre_word = cur_word
            cur_word = tf.expand_dims(tf.expand_dims(output[0,-1,:],axis=0),axis=0)
            
            #print(pre_word,cur_word)
            #print(cosine_similarity(tf.squeeze(pre_word,0),tf.squeeze(cur_word,0)))
            
            output_dim = tf.concat([output_dim,cur_word],axis=1)
            print(output_dim.shape,output.shape)
            
            result.append(cur_word)
            
            if len(result) > 93:
                stop_sign = True
            k+=1
        
        return result
        
        

class decoder(Model):
    def __init__(self):
        super(decoder,self).__init__()
        '''self.rnn = Sequential([
            Masking(mask_value=0.0),
            LSTM(512,input_shape=(94,768),return_sequences=True,return_state=True)
        ])'''
        self.mask = Masking(mask_value=0.0)
        self.rnn = LSTM(512,input_shape=(None,768),return_sequences=True,return_state=True)
        '''
        def lstm_model():
            inp = tf.keras.Input(shape=(94,768))
            out = LSTM(512,return_sequences=True, return_state=True)(inp)
            return models(inputs=inp,outputs=out)
        self.rnn = lstm_model()'''
    
    def call(self,enc_out,embs,txt_len):
        
        padded = []
        for i,k in enumerate(txt_len):
            tiled = tf.tile(tf.expand_dims(tf.expand_dims(enc_out[i],axis=0),axis=0),[1,len(k),1])
            pad = tf.convert_to_tensor(np.zeros((1,94-len(k),256)),tf.float32)
            tiled = tf.concat([tiled,pad],axis=1)
            padded.append(tiled)
        padded = tf.concat(padded,axis=0)
        cat = tf.concat([embs,padded],axis=2)
        cat = self.mask(cat)
        result,_,_ = self.rnn(cat)
        
        return result


In [178]:
enc = encoder()
dec = decoder()
model = seq2(enc,dec)
opt = tf.keras.optimizers.Adam(10e-3)

In [184]:
batch_mfcc.shape

(30, 40, 1021, 1)

In [191]:
enc(batch_mfcc)[0]

<tf.Tensor: shape=(256,), dtype=float32, numpy=
array([ 0.00000000e+00,  1.15157143e-34, -3.56603518e-18, -3.51215492e-20,
        1.31940450e-25,  1.36049617e-15, -1.00000000e+00, -3.44355961e-19,
        3.50765767e-15, -1.00000000e+00, -0.00000000e+00,  9.65265635e-34,
       -1.57946809e-29, -0.00000000e+00, -1.88600812e-19, -1.22576178e-31,
        1.69602797e-38,  1.00000000e+00,  0.00000000e+00, -3.91294153e-27,
        6.56244797e-37, -4.76049114e-41,  5.35369512e-14,  8.40779079e-45,
        9.35874730e-19,  1.22692985e-36,  1.00000000e+00,  0.00000000e+00,
       -5.30470397e-17, -1.30933368e-15, -4.35042460e-20, -1.00000000e+00,
        1.00000000e+00, -1.00000000e+00,  1.00404285e-22, -1.76586380e-29,
        7.61594176e-01,  4.27430492e-28,  9.23038059e-25, -2.11029748e-35,
        7.61594176e-01,  2.57781674e-25,  4.57523937e-21, -9.47081580e-41,
        2.37998862e-31, -3.90543865e-19,  1.16454328e-21, -7.31652540e-29,
       -1.59964015e-22, -1.51902443e-29, -7.61594176

In [179]:
#train

train_loss = tf.keras.metrics.Mean(name='train_loss')
df = display('start',display_id=True)

with tf.device(f'GPU:{0}'):
    for iter_num in range(100):
        for i in range(train_embs.shape[0]//30):
            batch_emb = train_embs[i*30:(i+1)*30]
            batch_mfcc = train_mfccs[i*30:(i+1)*30]
            batch_len = train_len[i*30:(i+1)*30]
            
            with tf.GradientTape() as tape:
                
                result = model(batch_mfcc,batch_emb,batch_len)
                loss=tf.reduce_mean(tf.reduce_mean(tf.math.sqrt(tf.reduce_sum(tf.math.pow(batch_emb-result,2),2)),1))
                train_loss(loss)
            grad = tape.gradient(loss,model.trainable_weights)
            opt.apply_gradients(zip(grad,model.trainable_weights))
            
        print(train_loss.result().numpy())
        df.update(tf.math.sqrt(tf.reduce_sum(tf.math.pow(batch_emb-result,2),2))[20])

<tf.Tensor: shape=(94,), dtype=float32, numpy=
array([0.36073387, 0.6729287 , 0.6325329 , 0.6562068 , 0.41682112,
       0.43409234, 0.6871867 , 0.4337584 , 0.66259   , 0.6653375 ,
       0.5618447 , 0.42240927, 0.5524554 , 0.68592554, 0.580096  ,
       0.41576064, 0.6463746 , 0.43530855, 0.637803  , 0.416999  ,
       0.699042  , 0.6525702 , 0.65170115, 0.67202   , 0.42004135,
       0.6567707 , 0.65478593, 0.60456645, 0.433784  , 0.6499607 ,
       0.6716782 , 0.6360808 , 0.43477705, 0.62636214, 0.41667286,
       0.4954039 , 0.56223285, 0.638811  , 0.43200466, 0.6248782 ,
       0.64727277, 0.495491  , 0.43148032, 0.6763049 , 0.62243074,
       0.5774987 , 0.6467609 , 0.5144158 , 0.43072677, 0.6429814 ,
       0.6385384 , 0.4335612 , 0.6289104 , 0.6663681 , 0.6353298 ,
       0.6401726 , 0.65672356, 0.60097015, 0.5767333 , 0.4318664 ,
       0.6613745 , 0.49935928, 0.5781455 , 0.6652991 , 0.41964108,
       0.6035451 , 0.32580376, 0.        , 0.        , 0.        ,
       0.      

0.17524664


KeyboardInterrupt: 

In [163]:
result[20]

<tf.Tensor: shape=(94, 512), dtype=float32, numpy=
array([[-0.02147044,  0.00081979, -0.0441295 , ..., -0.01005546,
         0.04903736,  0.04956301],
       [-0.00651785,  0.01091571,  0.01005996, ..., -0.01205452,
         0.00737195, -0.01954085],
       [-0.02829585,  0.03872909,  0.04408652, ..., -0.01198534,
        -0.02063626, -0.02882266],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)>

In [149]:
batch_emb[20]

array([[-0.02050252,  0.00103391, -0.04655671, ..., -0.01021517,
         0.04926512,  0.04633519],
       [-0.03076432,  0.04335005, -0.03050017, ...,  0.02118119,
        -0.01818778, -0.00318879],
       [ 0.03665725, -0.0431696 , -0.04205375, ..., -0.04627589,
         0.0222613 ,  0.04303093],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [174]:
np.diag(cosine_similarity(result[20],batch_emb[20]))

IndexError: list index out of range

In [180]:
result = model.predict(tf.expand_dims(train_mfccs[3],axis=0),sos_emb)
sent = emb_to_word(tf.squeeze(result))
print(sent)

(1, 1, 768)
(1, 2, 512) (1, 1, 512)
(1, 2, 768)
(1, 3, 512) (1, 2, 512)
(1, 3, 768)
(1, 4, 512) (1, 3, 512)
['<sos>', '<sos>', '<sos>', '<sos>']


In [152]:
encout = model.enc(tf.expand_dims(train_mfccs[3],axis=0))
ttt = tf.concat([expdim(expdim(sos_emb,0),0),expdim(encout,0)],axis=2)
rere,_,_ = model.dec.rnn(ttt)

In [153]:
sent = emb_to_word(tf.squeeze(rere,0))
print(sent)

['<sos>']


In [100]:
for i,k in enumerate(files):
    if i==3:
        print(files[k]['file_txt'])

['<sos>', 'his', 'progress', 'from', 'infancy', 'to', 'youth', 'and', 'manhood', 'was', 'marked', 'by', 'a', 'regular', 'increase', 'in', 'stature', 'and', 'wisdom', 'and', 'after', 'a', 'painful', 'agony', 'of', 'mind', 'and', 'body', 'he', 'expired', 'on', 'the', 'cross', '<eos>']


In [88]:
sent = emb_to_word(train_embs[3])
print(sent)

['<sos>', 'his', 'progress', 'from', 'infancy', 'to', 'youth', 'and', 'manhood', 'was', 'marked', 'by', 'a', 'regular', 'increase', 'in', 'stature', 'and', 'wisdom', 'and', 'after', 'a', 'painful', 'agony', 'of', 'mind', 'and', 'body', 'he', 'expired', 'on', 'the', 'cross', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']


In [84]:
cosine_similarity(expdim(train_embs[3][0],0),expdim(tf.squeeze(result[2]),0))

array([[-0.04609095]], dtype=float32)

In [80]:
train_embs[3,0].shape

(512,)

In [77]:
expdim(tf.squeeze(result[3]),0)

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[-1.75151348e-01,  8.72086908e-04, -7.78344423e-02,
        -1.01559650e-04,  2.28498946e-04, -1.99522838e-01,
        -2.51726800e-04,  4.77632973e-04,  2.92179078e-01,
        -1.64774861e-02,  5.35088480e-01,  1.93624511e-01,
        -2.53169332e-04,  2.14084890e-03,  3.87994982e-02,
         3.79973173e-01, -1.17874506e-03, -3.95371765e-02,
         2.69604176e-01, -4.56018627e-01, -5.01269184e-04,
         3.32064070e-02,  5.38751841e-01, -3.75390321e-01,
        -6.26827359e-01,  5.01064584e-03,  9.01191682e-02,
         1.73857401e-03,  2.76509821e-01,  1.99901331e-02,
        -4.86930072e-01,  2.10342882e-03,  3.66225606e-04,
         3.47866081e-02, -4.87887889e-01, -1.59370024e-02,
        -2.58146703e-01, -1.90740246e-02,  2.30877370e-01,
        -9.98292677e-03,  5.08802768e-04, -1.28934383e-01,
         2.17598841e-01, -1.86500326e-03,  2.58513074e-02,
         1.55694806e-03, -1.14755489e-01, -3.61822953e-04,
      

In [6]:
test(train_mfccs[0])

ValueError: Input 0 of layer conv2d is incompatible with the layer: expected ndim=4, found ndim=3. Full shape received: [40, 1021, 1]

In [228]:
test = tf.convert_to_tensor(np.expand_dims(librosa.feature.mfcc(wav,n_mfcc=40),axis=-1),tf.float32)
test = tf.transpose(test,[2,1,0])
print(test.shape)

(1, 126, 40)


In [30]:
mfccs_temp[:3]

[array([[[[-4.9675385e+02],
          [-4.9166388e+02],
          [-4.9072205e+02],
          ...,
          [ 0.0000000e+00],
          [ 0.0000000e+00],
          [ 0.0000000e+00]],
 
         [[ 2.8371893e+01],
          [ 3.5123100e+01],
          [ 3.8119247e+01],
          ...,
          [ 0.0000000e+00],
          [ 0.0000000e+00],
          [ 0.0000000e+00]],
 
         [[ 1.2961429e+01],
          [ 1.7783962e+01],
          [ 2.2991974e+01],
          ...,
          [ 0.0000000e+00],
          [ 0.0000000e+00],
          [ 0.0000000e+00]],
 
         ...,
 
         [[ 2.2770987e+00],
          [ 1.6555851e+00],
          [ 9.0551984e-01],
          ...,
          [ 0.0000000e+00],
          [ 0.0000000e+00],
          [ 0.0000000e+00]],
 
         [[ 4.3620820e+00],
          [ 1.9496366e+00],
          [ 9.7407675e-01],
          ...,
          [ 0.0000000e+00],
          [ 0.0000000e+00],
          [ 0.0000000e+00]],
 
         [[ 4.1309805e+00],
          [ 2.5641854e+00]

In [229]:
new_rnn = LSTM(512,input_shape=(None,40),return_sequences=True, return_state=True)
out,h,c = new_rnn(test)

In [230]:
c

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[ 1.49112556e-10,  5.43340269e-11, -5.58441341e-01,
        -1.00928879e+01,  2.11421609e+00, -2.75296807e+00,
         1.00000012e+00, -9.89174247e-02, -6.28049279e-11,
         7.80692825e+01,  1.21160555e+00, -1.25091872e-04,
         2.18741388e-05,  1.78486204e+00,  9.43991184e+00,
        -1.99343109e+01, -2.41631393e+01,  5.49382687e-01,
         1.68268542e-07,  2.38869593e-01,  1.68367885e-02,
        -6.65406644e-01, -1.77242506e+00, -1.97526751e-05,
         8.29644833e-08,  1.11458409e+00, -5.40364087e-10,
        -6.03983644e-03, -7.64443970e+00,  2.84235597e+00,
        -1.00000215e+00,  1.05299056e+00,  1.60268217e-03,
        -2.38807201e+01,  7.01753944e-09, -5.54915667e-02,
         2.98931713e-10,  8.22112083e+00,  1.47798026e-04,
         8.97779584e-01, -1.34507252e-03,  3.02408810e+01,
         3.63782578e+01,  9.99571383e-01,  1.21756864e+00,
        -5.25357027e-05, -6.45209468e-08,  1.98517041e-03,
      

(2433, 94, 512)

In [243]:



x_in = tf.keras.layers.Input((94,512))
x = LSTM(512,return_sequences=True,return_state=True)(x_in)
tdec = models(inputs=x_in,outputs=x)
opt = tf.keras.optimizers.Adam(10e-4)

for iter_num in range(100):
    for i in range(train_embs.shape[0]//30):
        batch_emb = train_embs[i*30:(i+1)*30]
        batch_len = train_len[i*30:(i+1)*30]
        with tf.GradientTape() as tape:
            git, _,_ = tdec(batch_emb)
            print(git.shape)
            loss = tf.reduce_sum(tf.reduce_mean(tf.math.sqrt(tf.reduce_sum(tf.math.pow((batch_emb - git),2),2)),1))
            print(loss)
        grad = tape.gradient(loss,tdec.trainable_variables)
        opt.apply_gradients(zip(grad,tdec.trainable_variables))

(30, 94, 512)
tf.Tensor(8.023993, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(7.1751356, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(6.1051884, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(4.475315, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(3.513566, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(3.1458747, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(3.9313812, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(3.6532183, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(6.6139607, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(5.368493, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(4.962204, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(4.4860444, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(4.504358, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(4.3563747, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(3.7815185, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(4.769044, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(4.8607254, shape=(), dt

(30, 94, 512)
tf.Tensor(3.78926, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(3.4396367, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.2741413, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.2217531, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.458003, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(3.2393289, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(3.3152082, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(3.9731014, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(5.5466304, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(4.4382687, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(3.8155549, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(3.4723926, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.564523, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.8749707, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.7309566, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.1276417, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.0285578, shape=(),

(30, 94, 512)
tf.Tensor(2.1466122, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(3.4396906, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.2941587, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.9609249, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.0488334, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.9952712, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.5551245, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.9590193, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.6443405, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.3902905, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.7291267, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.5605948, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(3.524314, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.5514076, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.9595494, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.0039144, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.0739894, shape=

(30, 94, 512)
tf.Tensor(1.9955006, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.6998131, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.6001939, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.7035106, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.4130285, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.6997043, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.502796, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.6695105, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.3727822, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.0547588, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.2295296, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.0297241, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(4.226772, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.000438, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.946908, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.9776667, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.180142, shape=(), 

(30, 94, 512)
tf.Tensor(2.0433967, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.2361796, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.2951243, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.3845845, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.5157826, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.2910185, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.3436453, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.6715953, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.8272352, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.2242994, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.8279407, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.1660966, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.9077075, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.7411884, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.680163, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(2.5102904, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(1.6038587, shape=

...

(30, 94, 512)
tf.Tensor(0.55635244, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.44613445, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.3716637, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.27126914, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.20756981, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.17933139, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.23186289, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.2259455, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.45624632, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.33521307, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.30257377, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.29674226, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.27753678, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.28748357, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.24960011, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.30963716, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.

(30, 94, 512)
tf.Tensor(0.14735115, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.22055066, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.2344124, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.21429509, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.30071536, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.21574846, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.16568328, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.16963823, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.18598054, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.25395775, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.22864711, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.27705818, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.2508027, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.17522375, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.16422473, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.17467451, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.

tf.Tensor(0.114872634, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.13961385, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.13086474, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.13803494, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.1907407, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.15819901, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.1833561, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.16163167, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.3118571, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.15754704, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.1566802, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.16053304, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.1742405, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.17232068, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.15076375, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.22660641, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.15376744, shape=

(30, 94, 512)
tf.Tensor(0.14832571, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.09500146, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.106914245, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.123673655, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.09231445, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.10842219, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.119808055, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.13749442, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.161017, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.13412017, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.09397566, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.132554, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.13003317, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.19246797, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.17200035, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0.11890744, shape=(), dtype=float32)
(30, 94, 512)
tf.Tensor(0

KeyboardInterrupt: 

In [244]:
print(expdim(sos_emb,0))
res,_,_ = tdec(expdim(expdim(sos_emb,0),0))

tf.Tensor(
[[-0.02050252  0.00103391 -0.04655671 -0.01166075  0.04239238 -0.00749804
  -0.03050377  0.02200724  0.04152847 -0.00033594  0.00146769  0.04834953
   0.03008405 -0.03558909  0.0377254   0.02654171  0.04034158  0.00121987
  -0.01320343 -0.03261739 -0.00089742  0.02038442  0.04140662  0.04047581
  -0.01321324 -0.04424632 -0.04615772  0.01930389  0.03872588 -0.00385382
  -0.02511166  0.03438803 -0.01388028 -0.02436901 -0.00027294 -0.02642584
  -0.04191203 -0.0448841  -0.02581958 -0.0012467  -0.00354906 -0.0234191
  -0.03354691 -0.04371642 -0.04315734  0.00201016  0.04830701 -0.0435622
   0.0326851   0.02308208 -0.02481061 -0.01683896 -0.02401296  0.03523275
   0.00141909  0.03272953 -0.00497752  0.00956595 -0.03158356 -0.00251736
   0.04969243 -0.04204947  0.01441059  0.04867392  0.01437552 -0.00279212
  -0.04715507  0.01696047  0.00337017 -0.02734038  0.02832029  0.02379251
   0.01649114 -0.01696166 -0.00623918  0.01144946  0.03992415 -0.04452405
  -0.04669755 -0.01119054  0.

In [245]:
res.shape

TensorShape([1, 1, 512])

In [246]:
emb_to_word(tf.squeeze(res,0))

['<sos>']